In [22]:
#visualization part
#import matplotlib.pyplot as plt
#import seaborn as sns


#basic libs
import pandas as pd
import numpy as np
import random as rnd

from sklearn.model_selection import cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train_=pd.read_csv('../train_allcols.csv')
validate_=pd.read_csv('../validate_allcols.csv')
#test=pd.read_csv('../testwDSM.csv')

train_.shape, validate_.shape, #test.shape

((301619, 62), (148560, 62))

In [23]:
#train.describe()
train = train_.sample(20000)
validate = validate_.sample(10000)
train_.shape, #validate_.shape, validate.head(2)

((301619, 62),)

In [24]:
train = train.query('SUB2 != 1')
validate = validate.query('SUB2 != 1')

train = train[train.SUB2.isin([2,3,4,5,7,10])]
validate = validate[validate.SUB2.isin([2,3,4,5,7,10])]
print train.shape
print train['SUB2'].value_counts()

(9609, 62)
4     3391
2     2794
3     1635
7      787
10     725
5      277
Name: SUB2, dtype: int64


In [25]:
# for sub2



X_train = train.drop(['SUB2'], axis=1)
Y_train = train["SUB2"]

from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif,chi2
#Selector_f = SelectPercentile(f_classif, percentile=25)
Selector_f = SelectKBest(f_classif, k=10)
Selector_f.fit(X_train,Y_train)

zipped = zip(X_train.columns.tolist(),Selector_f.scores_)
ans = sorted(zipped, key=lambda x: x[1])
for n,s in ans:    
    if 'FLG' in n: 
        pass
    else:
        print 'F-score: %3.2ft for feature %s' % (s,n)
                
#print X_train.columns.tolist()

F-score: 1.03t for feature CBSA10
F-score: 1.35t for feature DAYWAIT
F-score: 1.72t for feature PREG
F-score: 2.12t for feature VET
F-score: 3.26t for feature DETNLF
F-score: 3.87t for feature ARRESTS
F-score: 6.49t for feature PRIMPAY
F-score: 6.85t for feature ETHNIC
F-score: 8.00t for feature PRIMINC
F-score: 11.87t for feature LIVARAG
F-score: 12.31t for feature RACE
F-score: 12.93t for feature SUB3
F-score: 13.20t for feature REGION
F-score: 13.54t for feature GENDER
F-score: 13.71t for feature EMPLOY
F-score: 14.57t for feature MARSTAT
F-score: 15.37t for feature PSYPROB
F-score: 16.87t for feature STFIPS
F-score: 17.99t for feature FRSTUSE1
F-score: 20.12t for feature YEAR
F-score: 20.72t for feature CASEID
F-score: 21.39t for feature ROUTE3
F-score: 25.41t for feature SERVSETA
F-score: 27.98t for feature DETCRIM
F-score: 29.55t for feature DIVISION
F-score: 29.80t for feature EDUC
F-score: 30.02t for feature FREQ2
F-score: 32.25t for feature HLTHINS
F-score: 33.34t for feature 

In [26]:
# get the sorted feature list
import sys
for n, s in ans:
    
    if 'FLG' in n: 
        pass
    else:
        sys.stdout.write('\'%s\',' % (n))


'CBSA10','DAYWAIT','PREG','VET','DETNLF','ARRESTS','PRIMPAY','ETHNIC','PRIMINC','LIVARAG','RACE','SUB3','REGION','GENDER','EMPLOY','MARSTAT','PSYPROB','STFIPS','FRSTUSE1','YEAR','CASEID','ROUTE3','SERVSETA','DETCRIM','DIVISION','EDUC','FREQ2','HLTHINS','FRSTUSE3','SUB1','METHUSE','ROUTE1','FREQ1','PSOURCE','NOPRIOR','FREQ3','NUMSUBS','DSMCRIT','AGE','IDU','ROUTE2','ALCDRUG','FRSTUSE2',

In [27]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['SUB2',  'ROUTE2', 'FREQ2', 'FRSTUSE2', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'NUMSUBS'
             ]
retain_list = ['EMPLOY','GENDER','FREQ1','YEAR','EDUC','PSYPROB','PSOURCE','SERVSETA','DETCRIM',
               'REGION','NOPRIOR','DIVISION','DSMCRIT','ROUTE1','SUB1','AGE','IDU','SUB3','ROUTE3',
               'FREQ3','FRSTUSE3','FREQ2','FRSTUSE2']
X_train = train[retain_list]
Y_train = train["SUB2"]
X_validate = validate[retain_list]
Y_validate = validate["SUB2"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((9609, 23), (4738, 23))

In [28]:
print X_train.columns.tolist()

['EMPLOY', 'GENDER', 'FREQ1', 'YEAR', 'EDUC', 'PSYPROB', 'PSOURCE', 'SERVSETA', 'DETCRIM', 'REGION', 'NOPRIOR', 'DIVISION', 'DSMCRIT', 'ROUTE1', 'SUB1', 'AGE', 'IDU', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'FREQ2', 'FRSTUSE2']


In [29]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((9609, 184), (4738, 184))

In [30]:
# Logistic Regression
logreg = LogisticRegression(C=1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_validate, Y_validate) * 100, 2)
print acc_log


68.91


In [31]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
#Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_validate, Y_validate) * 100, 2)
print acc_perceptron


60.15


In [32]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


55.36


In [33]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=100, max_depth=20)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_validate, Y_validate) * 100, 2)
print acc_random_forest

#print cross_val_score(random_forest, X_validate, Y_validate)

65.58


In [34]:
# Stochastic Gradient Descent
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


62.71


In [35]:
# Linear SVC
linear_svc = LinearSVC(C=1)
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_validate, Y_validate) * 100, 2)
print acc_linear_svc


68.64


In [36]:
import xgboost as xgb
xgb_boost = xgb.XGBClassifier(seed=1850, n_jobs=-1)
xgb_boost.fit(X_train, Y_train) 
acc_xgb = round(xgb_boost.score(X_validate, Y_validate) * 100, 2)
print acc_xgb

67.18


In [37]:
print 'predict-top6-sub2-allcols-sample10000'
models = pd.DataFrame({
    'Model': [ 'Logistic Regression', 
              'Random Forest',  'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
               'XGBoost'],
    'Validate Set Score': [acc_log, 
              acc_random_forest,  acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_xgb]
    })
print models.sort_values(by='Validate Set Score', ascending=False)


predict-top6-sub2-allcols-sample10000
                        Model  Validate Set Score
0         Logistic Regression               68.91
4                  Linear SVC               68.64
5                     XGBoost               67.18
1               Random Forest               65.58
3  Stochastic Gradient Decent               62.71
2                  Perceptron               60.15
